In [83]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import seaborn as sns
from scipy import signal
from scipy.fft import fftshift, fft, fftfreq
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, 
NavigationToolbar2Tk)
from sklearn.utils import shuffle
import librosa as lib
from collections import Counter
import os
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, AveragePooling1D, Dropout, Dense, Input
from tensorflow.keras.models import Model, Sequential
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
import wave,sys
from scipy.io import wavfile
import warnings
warnings.filterwarnings('ignore')

import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter import *
import PIL
from PIL import Image, ImageEnhance
from PIL import Image, ImageTk
import tkcap
import pyautogui
import random
# create the root window
root = Tk()
root.title('Classification of Heart Sounds')
root.geometry('850x600')
root.configure(bg='red')
root.iconbitmap("C:/Users/Eagle/Desktop/img2.ico")
root.resizable(False, False)

load=Image.open("C:/Users/Eagle/Desktop/bg1.jpg")
load_1=load.resize((850,600),Image.ANTIALIAS)
render= ImageTk.PhotoImage(load_1,master=root)

img=Image.open("C:/Users/Eagle/Desktop/button2.jpg")
imgg=img.resize((115,115),Image.ANTIALIAS)
img_1= ImageTk.PhotoImage(imgg, master=root)

img2=Image.open("C:/Users/Eagle/Desktop/button3.jpg")
img_2=img2.resize((120,120),Image.ANTIALIAS)
img_3= ImageTk.PhotoImage(img_2,master=root)

img4=Image.open("C:/Users/Eagle/Desktop/but123.jpg")
imgg_4=img4.resize((120,120),Image.ANTIALIAS)
img_4= ImageTk.PhotoImage(imgg_4,master=root)
global filenames
global lib
filenames= None
#my_str= StringVar()
def select_files():
    global filenames
    global tail
    filetypes=[('PCG audio file', '*.wav*'),("All file","*")]
    filenames = fd.askopenfilename(title = "Select a File",filetypes=filetypes)
    if filenames =="": 
        filenames = None
        
        text.config(text= "Audio not selected")
    else:
        head, tail=os.path.split(filenames)
        ap = open(filenames, "r")
        ap.close()
        text.config(text="selected Audio==>"+tail)
    #if(filenames):
        #my_str.set(filenames)
    head, tail=os.path.split(filenames)
    global selected_path
    selected_path=tail
    global audd
    global aud
    global  lib
    audd=head+"/"+tail
    aud = audd
    dt , sr = lib.load(aud, sr=8000) # dt : raw array and sr:sampling rate
    dt = lib.to_mono(dt) # convert to mono channel if it is double
    dt = dt.astype('float32')

    # resampling frequecy rate into fix 2000 
    number_of_samples = round(len(dt) * float(8000) / sr) # total samples in full signals
    aud_resampled = signal.resample(dt, number_of_samples) # resample signal into calculated samples so getting 2000 sr. 

    b, a = signal.butter(10, [0.02,0.650], 'bandpass', analog=False) # 4rt order banpass butterworth filter
    aud_filtered = signal.filtfilt(b, a, aud_resampled) # apply filter to signal

    aud_filtered = aud_resampled

    input_length = 8000*4 # 10 seconds
    if len(aud_filtered) > input_length: # if audio len is greater then 10 sec
        max_offset = len(aud_filtered) - input_length   # max limit 
        offset = np.random.randint(max_offset)   # get a random position in the signal upto max limit
        aud_final = aud_filtered[offset:(input_length+offset)]  # get signal readings upto 10 sec 
    else:                                           # if signal len is less then 10 sec 
        dif = abs(input_length - len(aud_filtered))     # get require extended len 
        aud_final = np.pad(aud_filtered, (0,dif), mode = 'constant') # append zeros at end of signal reading equal to require extended len
    head, tail=os.path.split(filenames)
    n = random.randint(0,100)
    global audio_path
    audio_path=head+"patient_1"+str(n)+tail
    import soundfile as sf
    sf.write(audio_path, aud_final, 8000)

#def select_files():  
    
def Quit():
    root.destroy()

def Ai_Report():
    global tail 
    # Toplevel object which will
    # be treated as a new window
    ai = Toplevel(root)
 
    ai.geometry("800x650")
    ai.resizable(False, False)
    ai.iconbitmap("C:/Users/Eagle/Desktop/AI-1.ico")
    global audio_path
    global aud_1
    global Classes_prob
    global aud
    sample_rate,audio_data = wavfile.read(aud)
    audio_data = audio_data - np.mean(audio_data)
    audio_data = audio_data / np.max(audio_data)
    #print('Normalization Done')
    times = np.arange(len(audio_data))/float(sample_rate)
    fig = Figure(figsize = (4, 2),dpi = 100)
    plot1 = fig.add_subplot(111)
  
    # plotting the graph
    plot1.plot(times, audio_data)
    plot1.set_title('Time Vs. Amplitude')
    plot1.set_xlabel("Time")
    plot1.set_ylabel("Amplitude")
  
    # creating the Tkinter canvas
    # containing the Matplotlib figure
    canvas = FigureCanvasTkAgg(fig,master = ai)  
    canvas.draw()
    # placing the canvas on the Tkinter window
    canvas.get_tk_widget().place(x=300,y=190)
    
    
    fig_1= Figure(figsize = (4, 2),dpi = 100)
    plot_1 = fig_1.add_subplot(111)
  
    # plotting the graph
    plot_1.specgram(audio_data,NFFT=1024, Fs=sample_rate, noverlap=900)
    plot_1.set_title('Frequency Domain')
    plot_1.set_ylabel("Frequency [Hz]")
  
    # creating the Tkinter canvas
    # containing the Matplotlib figure
    canvas = FigureCanvasTkAgg(fig_1,master = ai)  
    canvas.draw()
    # placing the canvas on the Tkinter window
    canvas.get_tk_widget().place(x=300,y=440)
    
    aud_1 =audio_path
    dd,s=lib.load(aud_1,sr=8000)
    dd = dd.reshape(1,8000,4)
    model = tf.keras.models.load_model(r"C:\Users\Eagle\Downloads\conv_model1 (3).h5")
    class1 = model.predict(dd)
    classes=np.argmax(class1)
    class_1=class1[0]
    cls_0=class_1[0]*100
    clss_0="{:.2f}".format(cls_0)
    cls_1=class_1[1]*100
    clss_1="{:.2f}".format(cls_1)
    class_A=clss_0+'%'" Normal"" ",clss_1+'%'" Abnormal"
    #if class_1[1] >= 0.5:
        #cl_1=clss_1,"%",'Detected Abnormal \n Input_Signal',tail
    #else:
        #cl_1=clss_0,"%",'Detected Normal \n input_Signal',tail
    #global ent9
    #ent9=Entry(ai,width=20)
    #ent9.place(x=100,y=620)
    head, tail=os.path.split(filenames)
    global save_path
    save_path = head +"/"+tail+".jpg"

    global nr
    nr=0
    nr = random.randint(0,100)
    def fSave():
        #global ent9
        #path=ent9.get()
        global nr
        nr += 1
        global save_path
        save_path = head +"/"+tail+"_"+str(nr) + ".jpg"
        cap = tkcap.CAP(ai)  # screenshot of ai window
        screnshot=cap.capture(save_path)
        #cap = tkcap.CAP(ai)  # screenshot of ai window
        #ai.bind('<Control-g>', lambda: cap.capture())
        open_image = Image.open(save_path)
        color = ImageEnhance.Color(open_image)
        imgg=color.enhance(2.5)
        contrast = ImageEnhance.Contrast(imgg)
        im=contrast.enhance(1.5) #.save('contrast.jpg')
        
        
        box = (3, 30, 795, 675) #left upper right lower
        cropped_image = im.crop(box)
        
        #img=open_image.resize(("3508x2480"),Image.ANTIALIAS)
        #convert_image = img.convert('RGB')
 
        # Output Pdf Path
        outputpdfPath =head +"/"+tail+"_"+str(nr)+ ".pdf"
 
        # Saving the pdf
        cropped_image.save(outputpdfPath)
        if outputpdfPath =="": 
            outputpdfPath = None
            text_1.config(text= "Pdf not Saved")
        else:
            ap = open(outputpdfPath, "r")
            ap.close()
            text_1.config(text="Pdf saved")

     
    # sets the title of the
    # Toplevel widget
    ai.title("AI Report")
    lb1=Label(ai, text="Patient Name:",font=('Helvetica',10,"bold"),fg="black").place(x=5,y=5)
    ent1=Entry(ai,font=('Helvetica',10,"bold"),width=20).place(x=100,y=5)
    
    lb2=Label(ai, text="Age(Year):",font=('Helvetica',10,"bold"),fg="black").place(x=250,y=5)
    ent2=Entry(ai,font=('Helvetica',10,"bold"),width=10).place(x=325,y=5)
    
    lb3=Label(ai, text="Location:",font=('Helvetica',10,"bold"),fg="black").place(x=405,y=5)
    ent3=Entry(ai,font=('Helvetica',10,"bold"),width=50).place(x=470,y=5)
    
    lb_3=Label(ai, text="History:",font=('Helvetica',10,"bold"),fg="black").place(x=5,y=30)
    text1=Text(ai,font=('Helvetica',10,"bold"),width=30,height=15).place(x=20,y=60)
    
    lb4=Label(ai,text="Probabilistic AI Result",font=('Helvetica',15,"bold"),fg="blue").place(x=350,y=30)
    lb_4=Label(ai,text=class1,font=('Helvetica',15,"bold"),fg="black",bd=0).place(x=350,y=60)
    lb_5=Label(ai,text=class_A,font=('Helvetica',15,"bold"),fg="black",bd=0).place(x=300,y=90)
    lb_8=Label(ai,text="selected Signal=>"+selected_path,font=('Helvetica',15,"bold"),fg="black",bd=0).place(x=300,y=120)
    
    lb_6=Label(ai,text="PCG",font=('Helvetica',15,"bold"),fg="blue",bd=0).place(x=420,y=160)
    #Frame(ai,bg="gray",bd=2,width=400,height=200).place(x=320,y=210)
    
    lb6=Label(ai,text="Spectrogram",font=('Helvetica',15,"bold"),fg="blue").place(x=380,y=410)
    #Frame(ai,bg="gray",bd=2,width=400,height=200).place(x=320,y=440)
    
    lb7=Label(ai,text="Comments:",font=('Helvetica',10,"bold"),fg="black").place(x=5,y=320)
    text=Text(ai,font=('Helvetica',10,"bold"),width=30,height=15).place(x=20,y=350)
    bt = Button(ai,text="Save as PDF",bd=2,command=fSave,font=('Helvetica',10,"bold"),bg="white",fg="black",activebackground="white")
    bt.place(x=5, y=615)
    text_1= Label(ai, text= "",font=('Helvetica',10,"bold"), fg="black", bg="white",borderwidth=0)
    text_1.place(x=100,y=620)
    # sets the geometry of toplevel
   
 
    # A Label widget to show in toplevel
    

lb=Label(root, image=render,bg="cyan").place(x=0, y=0, relwidth=1, relheight=1)

Label(root,text ="Automatic Paediatric Heart Abnormality Detection System",borderwidth=0,font=("Helvetica",15,"bold"),fg="black",bg="white").pack(pady=10,anchor=NW)

# open button
b1 = Button(root,image=img_1,bd=0,width=110,height=110,bg="white",activebackground="white",command=select_files)
b1.place(x=30, y=50)
text= Label(root, text= "",font=('Helvetica',10,"bold"), fg="black", bg="white",borderwidth=0)
text.place(x=175,y=100)

b4 = Button(root,image=img_3,command=Ai_Report,width=110,height=110,bd=0,activebackground="white",bg="white")
b4.place(x=30, y=170)

b5 = Button(root,image=img_4,command= Quit,bd=0,width=110,height=110,bg="white")
b5.place(x=30,y=500)

root.mainloop()

1/1 [==============================] - 0s 215ms/step
